###  Created by Luis A. Sanchez-Perez (l.alejandro.2011@gmail.com).
<p><span style="color:green"><b>Copyright &#169;</b> Do not distribute or use without authorization from author.</span></p>

In [1]:
import pathlib
import pandas as pd

In [2]:
DATASETS = pathlib.Path('/media/alejand/DatasetsT7/datasets')

In [3]:
# Loads ratings
ratings = pd.read_csv(DATASETS / 'recommender/movies/ml-20m/ratings.csv').drop(columns=['timestamp'])
ratings = ratings.set_index(['userId', 'movieId'])
ratings.head(10)

rating
userId movieId        
1      2           3.5
       29          3.5
       32          3.5
       47          3.5
       50          3.5
       112         3.5
       151         4.0
       223         4.0
       253         4.0
       260         4.0

In [4]:
# Reads movies
movies = pd.read_csv('encoded_movies.csv', index_col='movieId')
# Gets a list of genres as loaded from file (in the same order)
index_to_genre = list(movies.columns[1:-1])

In [5]:
# Merging movies features with user reviews (memory intensive operation!!)
movies_rated = ratings.merge(movies, left_index=True, right_index=True)
movies_rated.drop(columns=['title', 'year'], inplace=True)
movies_rated.head(10)

rating  Adventure  Animation  Children  Comedy  Fantasy  \
userId movieId                                                            
1      2           3.5          1          0         1       0        1   
       29          3.5          1          0         0       0        1   
       32          3.5          0          0         0       0        0   
       47          3.5          0          0         0       0        0   
       50          3.5          0          0         0       0        0   
       112         3.5          1          0         0       1        0   
       151         4.0          0          0         0       0        0   
       223         4.0          0          0         0       1        0   
       253         4.0          0          0         0       0        0   
       260         4.0          1          0         0       0        0   

                Romance  Drama  Action  Crime  ...  Horror  Mystery  Sci-Fi  \
userId movieId                                 ...                            
1      2              0      0       0      0  ...       0        0       0   
       29             0      1       0      0  ...       0        1       1   
       32             0      0       0      0  ...       0        1       1   
       47             0      0       0      0  ...       0        1       0   
       50             0      0       0      1  ...       0        1       0   
       112            0      0       1      1  ...       0        0       0   
       151            1      1       1      0  ...       0        0       0   
       223            0      0       0      0  ...       0        0       0   
       253            0      1       0      0  ...       1        0       0   
       260            0      0       1      0  ...       0        0       1   

                IMAX  Documentary  War  Musical  Western  Film-Noir  \
userId movieId                                                        
1      2           0            0    0        0        0          0   
       29          0            0    0        0        0          0   
       32          0            0    0        0        0          0   
       47          0            0    0        0        0          0   
       50          0            0    0        0        0          0   
       112         0            0    0        0        0          0   
       151         0            0    1        0        0          0   
       223         0            0    0        0        0          0   
       253         0            0    0        0        0          0   
       260         0            0    0        0        0          0   

                (no genres listed)  
userId movieId                      
1      2                         0  
       29                        0  
       32                        0  
       47                        0  
       50                        0  
       112                       0  
       151                       0  
       223                       0  
       253                       0  
       260                       0  

[10 rows x 21 columns]

In [6]:
# # Generates a vector per user that has rated a movie (another version)
# def compute_embedding(group):
#     values = group.values
#     rating = values[:, 0].reshape(-1,1)
#     encoding = rating * values[:, 1:]
#     return encoding.sum(axis=0) / encoding.sum()
# embeddings = movies_rated.groupby(level='userId').apply(compute_embedding)

In [7]:
# Generates a vector per user that has rated a movie
users = {}
grouped = movies_rated.groupby(level='userId')
for key, data in  grouped:
    values = data.values
    rating = values[:, 0].reshape(-1,1)
    encoding = rating * values[:, 1:]
    users[key] = encoding.sum(axis=0) / encoding.sum()

In [8]:
# Converts to csv to easily store into csv
vectors = pd.DataFrame.from_dict(users, orient='index', columns=index_to_genre)
vectors.index.name = 'userId'
vectors.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
userId,,,,,,,,,,,,,,,,,,,,
1,0.143005,0.018878,0.035428,0.079131,0.135247,0.022498,0.083786,0.127230,0.041376,0.081717,0.087148,0.033618,0.076804,0.004396,0.000000,0.017067,0.005689,0.006982,0.000000,0.0
2,0.130573,0.004777,0.004777,0.062102,0.003185,0.036624,0.117834,0.140127,0.007962,0.128981,0.101911,0.028662,0.168790,0.004777,0.000000,0.027070,0.009554,0.014331,0.007962,0.0
3,0.103027,0.007324,0.020996,0.103027,0.041992,0.031738,0.119629,0.122559,0.043945,0.104004,0.061523,0.023438,0.181641,0.000000,0.001953,0.013672,0.011719,0.006348,0.001465,0.0
4,0.079585,0.027682,0.051903,0.134948,0.038062,0.048443,0.103806,0.159170,0.065744,0.155709,0.000000,0.027682,0.051903,0.000000,0.000000,0.013841,0.027682,0.013841,0.000000,0.0
5,0.121951,0.035944,0.059050,0.125802,0.052632,0.080873,0.145058,0.103979,0.037227,0.083440,0.003851,0.008986,0.059050,0.019255,0.000000,0.005135,0.044929,0.012837,0.000000,0.0


In [9]:
# Saves to file
vectors.to_csv('users_embeddings.csv')